# SQLAlchemy 1.4 / 2.0 Tutorial

## Establishing Connectivity - the Engine

In [2]:
import os.path
import yaml
from sqlalchemy import create_engine, text

config_path = os.path.expanduser('~\\.sqlalchemy\\config.yaml')

with open(config_path) as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

db_path = (
    f"postgresql://{config['username']}:{config['password']}"
    f"@{config['host']}:{config['port']}/{config['db_name']}"
)
engine = create_engine(db_path, echo=True, future=True)

with engine.connect() as conn:
    result = conn.execute(text("select 'hello world'"))
    print(result.all())


2021-11-22 08:09:10,899 INFO sqlalchemy.engine.Engine select version()
2021-11-22 08:09:10,899 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-11-22 08:09:10,909 INFO sqlalchemy.engine.Engine select current_schema()
2021-11-22 08:09:10,909 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-11-22 08:09:10,910 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2021-11-22 08:09:10,911 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-11-22 08:09:10,912 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-22 08:09:10,913 INFO sqlalchemy.engine.Engine select 'hello world'
2021-11-22 08:09:10,913 INFO sqlalchemy.engine.Engine [generated in 0.00094s] {}
[('hello world',)]
2021-11-22 08:09:10,914 INFO sqlalchemy.engine.Engine ROLLBACK


## Working with Transactions and the DBAPI

### Committing Changes

In [15]:
# commit as you go
with engine.connect() as conn:
    conn.execute(text("CREATE TABLE FUNC( x integer, y integer )"))
    conn.execute(text("INSERT INTO FUNC (x,y) VALUES (:x, :y)"),
                 [{"x": 1, "y": 1}, {"x": 2, "y": 4}])
    conn.commit()

# begin once
with engine.begin() as conn:
    conn.execute(text("INSERT INTO FUNC (x,y) VALUES (:x, :y)"),
                 [{"x": 6, "y": 8}, {"x": 9, "y": 10}])


2021-11-18 15:15:22,626 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-18 15:15:22,627 INFO sqlalchemy.engine.Engine CREATE TABLE FUNC( x integer, y integer )
2021-11-18 15:15:22,627 INFO sqlalchemy.engine.Engine [generated in 0.00121s] {}
2021-11-18 15:15:22,652 INFO sqlalchemy.engine.Engine INSERT INTO FUNC (x,y) VALUES (%(x)s, %(y)s)
2021-11-18 15:15:22,653 INFO sqlalchemy.engine.Engine [generated in 0.00099s] ({'x': 1, 'y': 1}, {'x': 2, 'y': 4})
2021-11-18 15:15:22,655 INFO sqlalchemy.engine.Engine COMMIT
2021-11-18 15:15:22,658 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-18 15:15:22,659 INFO sqlalchemy.engine.Engine INSERT INTO FUNC (x,y) VALUES (%(x)s, %(y)s)
2021-11-18 15:15:22,659 INFO sqlalchemy.engine.Engine [cached since 0.007468s ago] ({'x': 6, 'y': 8}, {'x': 9, 'y': 10})
2021-11-18 15:15:22,663 INFO sqlalchemy.engine.Engine COMMIT


### Basics of Statement Execution

In [17]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x,y FROM FUNC;"))
    for row in result:
        print(f"x: {row.x} y: {row.y}")

    result = conn.execute(text("SELECT x,y FROM FUNC;"))
    for dict_row in result.mappings():
        x = dict_row['x']
        y = dict_row['y']
        print(x, y)

    result = conn.execute(text("SELECT x,y FROM FUNC where y > :y"), {"y": 2})
    for row in result:
        print(f"x: {row.x} y: {row.y}")

    conn.execute(text("INSERT INTO FUNC(x,y) VALUES (:x, :y)"),
                 [{"x": 11, "y": 12}, {"x": 13, "y": 14}])
    conn.commit()

    # bundling parameters
    stmt = text(
        "SELECT x, y FROM FUNC WHERE y > :y ORDER BY x,y"
        ).bindparams(y=6)
    result = conn.execute(stmt)
    for row in result:
        print(f"x: {row.x} y: {row.y}")


2021-11-18 15:56:35,409 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-18 15:56:35,410 INFO sqlalchemy.engine.Engine SELECT x,y FROM FUNC;
2021-11-18 15:56:35,410 INFO sqlalchemy.engine.Engine [generated in 0.00166s] {}
x: 1 y: 1
x: 2 y: 4
x: 6 y: 8
x: 9 y: 10
2021-11-18 15:56:35,412 INFO sqlalchemy.engine.Engine SELECT x,y FROM FUNC;
2021-11-18 15:56:35,413 INFO sqlalchemy.engine.Engine [cached since 0.004293s ago] {}
1 1
2 4
6 8
9 10
2021-11-18 15:56:35,415 INFO sqlalchemy.engine.Engine SELECT x,y FROM FUNC where y > %(y)s
2021-11-18 15:56:35,415 INFO sqlalchemy.engine.Engine [generated in 0.00051s] {'y': 2}
x: 2 y: 4
x: 6 y: 8
x: 9 y: 10
2021-11-18 15:56:35,420 INFO sqlalchemy.engine.Engine INSERT INTO FUNC(x,y) VALUES (%(x)s, %(y)s)
2021-11-18 15:56:35,421 INFO sqlalchemy.engine.Engine [generated in 0.00087s] ({'x': 11, 'y': 12}, {'x': 13, 'y': 14})
2021-11-18 15:56:35,422 INFO sqlalchemy.engine.Engine COMMIT
2021-11-18 15:56:35,423 INFO sqlalchemy.engine.Engine BEGIN (impl

### Executing with an ORM Session

In [18]:
from sqlalchemy.orm import Session
stmt = text(
    "SELECT x, y FROM FUNC WHERE y > :y ORDER BY x, y"
).bindparams(y=6)

with Session(engine) as session:
    result = session.execute(stmt)
    for row in result:
        print(f"x: {row.x} y: {row.y}")

    result = session.execute(
        text("UPDATE FUNC SET y=:y WHERE x=:x"),
        [{"x": 9, "y": 11}, {"x": 13, "y": 15}]
    )
    session.commit()  # To save, you should commit


2021-11-18 16:02:08,297 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-18 16:02:08,313 INFO sqlalchemy.engine.Engine SELECT x, y FROM FUNC WHERE y > %(y)s ORDER BY x, y
2021-11-18 16:02:08,313 INFO sqlalchemy.engine.Engine [generated in 0.00039s] {'y': 6}
x: 6 y: 8
x: 9 y: 10
x: 11 y: 12
x: 13 y: 14
2021-11-18 16:02:08,313 INFO sqlalchemy.engine.Engine UPDATE FUNC SET y=%(y)s WHERE x=%(x)s
2021-11-18 16:02:08,313 INFO sqlalchemy.engine.Engine [generated in 0.00036s] ({'y': 11, 'x': 9}, {'y': 15, 'x': 13})
2021-11-18 16:02:08,319 INFO sqlalchemy.engine.Engine COMMIT


## Working with Database Metadata

### Setting up Metadata with Table objects

In [5]:
from sqlalchemy import MetaData
metadata_obj = MetaData()  # a set of Table object

print(metadata_obj)

MetaData()


In [6]:
from sqlalchemy import Table, Column, Integer, String
user_table = Table(
    "user_account",  # table name
    metadata_obj,  # 'metadata_obj' has this table.
    Column('id', Integer, primary_key=True),
    Column('name', String(30)),
    Column('fullname', String)
)


### Declaring Simple Constraints

In [7]:
user_table.primary_key

PrimaryKeyConstraint(Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False))

In [8]:
from sqlalchemy import ForeignKey
#  ForeignKey : we can omit the datatype for this column
address_table = Table(
    "address",
    metadata_obj,
    Column('id', Integer, primary_key=True),
    Column('user_id', ForeignKey('user_account.id'), nullable=False),
    Column('email_address', String, nullable=False)
)


### Emitting DDL to the Database

In [36]:
# create
metadata_obj.create_all(engine)

2021-11-19 08:51:33,161 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-19 08:51:33,163 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2021-11-19 08:51:33,164 INFO sqlalchemy.engine.Engine [cached since 2231s ago] {'name': 'user_account'}
2021-11-19 08:51:33,166 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2021-11-19 08:51:33,166 INFO sqlalchemy.engine.Engine [cached since 2231s ago] {'name': 'address'}
2021-11-19 08:51:33,168 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id SERIAL NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2021-11-19 08:51:33,168 INFO sqlalchemy.engine.Engine [no key 0.00064s] {}
2021-11-19 08:51:33,182 INFO sqlalchemy.engine.Engine 
CREATE TABLE address (
	id SERIAL NOT NUL

In [36]:
# delete
metadata_obj.drop_all(engine)

2021-11-18 16:22:16,961 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-18 16:22:16,962 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2021-11-18 16:22:16,963 INFO sqlalchemy.engine.Engine [cached since 10.58s ago] {'name': 'user_account'}
2021-11-18 16:22:16,968 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2021-11-18 16:22:16,968 INFO sqlalchemy.engine.Engine [cached since 10.58s ago] {'name': 'address'}
2021-11-18 16:22:16,969 INFO sqlalchemy.engine.Engine 
DROP TABLE address
2021-11-18 16:22:16,971 INFO sqlalchemy.engine.Engine [no key 0.00182s] {}
2021-11-18 16:22:16,976 INFO sqlalchemy.engine.Engine 
DROP TABLE user_account
2021-11-18 16:22:16,977 INFO sqlalchemy.engine.Engine [no key 0.00057s] {}
2021-11-18 16:22:16,978 INFO sqla

### Defining Table Metadata with the ORM

In [37]:
# Setting up the Registry
from sqlalchemy.orm import registry
mapper_registry = registry()
mapper_registry.metadata
Base = mapper_registry.generate_base()  # declarative base


In [38]:
# Declaring Mapped Classes (using base)
from sqlalchemy.orm import relationship


class User(Base):
    __tablename__ = 'user_account'

    id = Column(Integer, primary_key=True)
    name = Column(String(30))
    fullname = Column(String)
    addresses = relationship("Address", back_populates="user")
    # back_populates: Establishing a relationship in both direction.

    def __repr__(self):
        return (
            f"User(id={self.id!r}, name={self.name!r}, "
            f"fullname={self.fullname!r})"
        )
        # !r : repr(). a formal string that the interpreter can recognize


class Address(Base):
    __tablename__ = 'address'

    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('user_account.id'))

    user = relationship("User", back_populates="addresses")

    def __repr__(self):
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"


In [26]:
User.__table__

Table('user_account', MetaData(), Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False), Column('name', String(length=30), table=<user_account>), Column('fullname', String(), table=<user_account>), schema=None)

In [39]:
# Other Mapped Class Details
sandy = User(name="sandy", fullname="Sandy Cheeks")  # create a instance
sandy  # __repr__() is used

User(id=None, name='sandy', fullname='Sandy Cheeks')

In [44]:
# Emitting DDL to the database. (three ways to emit DDL)
# 1) MetaData.create_all(engine)
# 2) mapper_registry.metadata.create_all(engine)
# 3) Base.metadata.create_all(engine)


In [46]:
# Combining Core Table Declarations with ORM Declarative
# class User(Base):
#    __table__ = user_table  # we don't have to write column infos.

### Table Reflection

In [12]:
# Load table from existing database as ORM object
# (not declaring table directly)
funcrefl = Table("func", metadata_obj, autoload_with=engine)


2021-11-19 08:11:23,967 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-19 08:11:23,968 INFO sqlalchemy.engine.Engine 
            SELECT c.oid
            FROM pg_catalog.pg_class c
            LEFT JOIN pg_catalog.pg_namespace n ON n.oid = c.relnamespace
            WHERE (pg_catalog.pg_table_is_visible(c.oid))
            AND c.relname = %(table_name)s AND c.relkind in
            ('r', 'v', 'm', 'f', 'p')
        
2021-11-19 08:11:23,969 INFO sqlalchemy.engine.Engine [generated in 0.00143s] {'table_name': 'func'}
2021-11-19 08:11:23,971 INFO sqlalchemy.engine.Engine 
            SELECT a.attname,
              pg_catalog.format_type(a.atttypid, a.atttypmod),
              (
                SELECT pg_catalog.pg_get_expr(d.adbin, d.adrelid)
                FROM pg_catalog.pg_attrdef d
                WHERE d.adrelid = a.attrelid AND d.adnum = a.attnum
                AND a.atthasdef
              ) AS DEFAULT,
              a.attnotnull,
              a.attrelid as table_oid,


In [13]:
funcrefl

Table('func', MetaData(), Column('x', INTEGER(), table=<func>), Column('y', INTEGER(), table=<func>), schema=None)

## Working with Data

### Inserting Rows with Core

In [40]:
from sqlalchemy import insert
stmt = insert(user_table).values(
    name='spongebob', fullname="Spongebob Squarepants"
)
print(stmt)
complied = stmt.compile()
complied.params

with engine.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()
    print(result.inserted_primary_key)  # tuple type


INSERT INTO user_account (name, fullname) VALUES (:name, :fullname)
2021-11-19 08:51:53,126 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-19 08:51:53,128 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (%(name)s, %(fullname)s) RETURNING user_account.id
2021-11-19 08:51:53,128 INFO sqlalchemy.engine.Engine [generated in 0.00191s] {'name': 'spongebob', 'fullname': 'Spongebob Squarepants'}
2021-11-19 08:51:53,133 INFO sqlalchemy.engine.Engine COMMIT
(1,)


In [41]:
with engine.connect() as conn:
    result = conn.execute(
        insert(user_table),
        [
            {"name": "sandy", "fullname": "Sandy Cheeks"},
            {"name": "patrick", "fullname": "Patrick Star"}
        ]
    )
    conn.commit()

2021-11-19 08:51:55,570 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-19 08:51:55,571 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (%(name)s, %(fullname)s)
2021-11-19 08:51:55,572 INFO sqlalchemy.engine.Engine [generated in 0.00159s] ({'name': 'sandy', 'fullname': 'Sandy Cheeks'}, {'name': 'patrick', 'fullname': 'Patrick Star'})
2021-11-19 08:51:55,573 INFO sqlalchemy.engine.Engine COMMIT


In [42]:
# insert using subquery
from sqlalchemy import select, bindparam
scalar_subq = (
    select(user_table.c.id).
    where(user_table.c.name == bindparam('username')).
    scalar_subquery()
)

print(scalar_subq)

with engine.connect() as conn:
    result = conn.execute(
        insert(address_table).values(user_id=scalar_subq),
        [
            {"username": 'spongebob', "email_address": "spongebob@sqlalchemy.org"},
            {"username": 'sandy', "email_address": "sandy@sqlalchemy.org"},
            {"username": 'sandy', "email_address": "sandy@squirrelpower.org"},
        ]
    )
    conn.commit()


(SELECT user_account.id 
FROM user_account 
WHERE user_account.name = :username)
2021-11-19 08:51:58,750 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-19 08:51:58,751 INFO sqlalchemy.engine.Engine INSERT INTO address (user_id, email_address) VALUES ((SELECT user_account.id 
FROM user_account 
WHERE user_account.name = %(username)s), %(email_address)s)
2021-11-19 08:51:58,751 INFO sqlalchemy.engine.Engine [generated in 0.00160s] ({'username': 'spongebob', 'email_address': 'spongebob@sqlalchemy.org'}, {'username': 'sandy', 'email_address': 'sandy@sqlalchemy.org'}, {'username': 'sandy', 'email_address': 'sandy@squirrelpower.org'})
2021-11-19 08:51:58,753 INFO sqlalchemy.engine.Engine COMMIT


In [44]:
from sqlalchemy import select, insert
# insert from select
select_stmt = select(user_table.c.id, user_table.c.name + "@aol.com")
print(select_stmt)
insert_stmt = insert(address_table).from_select(
    ["user_id", "email_address"], select_stmt
)
print(insert_stmt)


SELECT user_account.id, user_account.name || :name_1 AS anon_1 
FROM user_account
INSERT INTO address (user_id, email_address) SELECT user_account.id, user_account.name || :name_1 AS anon_1 
FROM user_account


In [48]:
# insert...Returning ('RETURNING' is supported by insert, update, delete)
insert_stmt = insert(address_table).returning(
    address_table.c.id, address_table.c.email_address
)
print(insert_stmt)

select_stmt = select(user_table.c.id, user_table.c.name + "@aol.com")
insert_stmt = insert(address_table).from_select(
    ["user_id", "email_address"], select_stmt
)
print(insert_stmt.returning(address_table.c.id, address_table.c.email_address))


INSERT INTO address (id, user_id, email_address) VALUES (:id, :user_id, :email_address) RETURNING address.id, address.email_address
INSERT INTO address (user_id, email_address) SELECT user_account.id, user_account.name || :name_1 AS anon_1 
FROM user_account RETURNING address.id, address.email_address


### Selecting Rows with Core or ORM

#### The select() SQL Expression Construct

In [51]:
from sqlalchemy import select
stmt = select(user_table).where(user_table.c.name == 'spongebob')
print(stmt)

with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(row)


SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1
2021-11-19 10:03:04,484 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-19 10:03:04,485 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = %(name_1)s
2021-11-19 10:03:04,485 INFO sqlalchemy.engine.Engine [cached since 151.8s ago] {'name_1': 'spongebob'}
(1, 'spongebob', 'Spongebob Squarepants')
2021-11-19 10:03:04,486 INFO sqlalchemy.engine.Engine ROLLBACK


In [55]:
from sqlalchemy.orm import Session
# using the ORM : ORM-mapped instances should be returned
stmt = select(User).where(User.name == 'spongebob')
with Session(engine) as session:
    for row in session.execute(stmt):
        print(row)


2021-11-19 10:07:37,078 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-19 10:07:37,079 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = %(name_1)s
2021-11-19 10:07:37,080 INFO sqlalchemy.engine.Engine [cached since 188.9s ago] {'name_1': 'spongebob'}
(User(id=1, name='spongebob', fullname='Spongebob Squarepants'),)
2021-11-19 10:07:37,081 INFO sqlalchemy.engine.Engine ROLLBACK


#### Setting the COLUMNS and FROM clause

In [67]:
print(select(user_table))  # User table is 'table object'
print(select(user_table.c.name, user_table.c.fullname))  # Core approach


SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
SELECT user_account.name, user_account.fullname 
FROM user_account


In [68]:
# selecting ORM entites and columns
print(select(User))
row = session.execute(select(User)).first()
row  # row objects which contain instances of the User class

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2021-11-19 10:20:32,509 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2021-11-19 10:20:32,511 INFO sqlalchemy.engine.Engine [cached since 344.5s ago] {}


(User(id=1, name='spongebob', fullname='Spongebob Squarepants'),)

In [63]:
row[0]

User(id=1, name='spongebob', fullname='Spongebob Squarepants')

In [65]:
print(select(User.name, User.fullname))

SELECT user_account.name, user_account.fullname 
FROM user_account


In [66]:
row = session.execute(select(User.name, User.fullname)).first()
row

2021-11-19 10:18:24,536 INFO sqlalchemy.engine.Engine SELECT user_account.name, user_account.fullname 
FROM user_account
2021-11-19 10:18:24,536 INFO sqlalchemy.engine.Engine [generated in 0.00098s] {}


('spongebob', 'Spongebob Squarepants')

In [73]:
session.execute(
    select(User.name, Address).  # mixing the attributes and full entities
    where(User.id == Address.user_id).
    order_by(Address.id)
).all()


2021-11-19 10:26:20,706 INFO sqlalchemy.engine.Engine SELECT user_account.name, address.id, address.email_address, address.user_id 
FROM user_account, address 
WHERE user_account.id = address.user_id ORDER BY address.id
2021-11-19 10:26:20,707 INFO sqlalchemy.engine.Engine [cached since 215.4s ago] {}


[('spongebob', Address(id=1, email_address='spongebob@sqlalchemy.org')),
 ('sandy', Address(id=2, email_address='sandy@sqlalchemy.org')),
 ('sandy', Address(id=3, email_address='sandy@squirrelpower.org'))]

In [77]:
# selecting from "labeled" SQL Expressions : like "AS ~"
from sqlalchemy import func, cast
stmt = (
    select(
        ("Username: " + user_table.c.name).label("username"),
    ).order_by(user_table.c.name)
)

with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(f"{row.username}")  # we can call using labels


2021-11-19 10:29:44,635 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-19 10:29:44,636 INFO sqlalchemy.engine.Engine SELECT %(name_1)s || user_account.name AS username 
FROM user_account ORDER BY user_account.name
2021-11-19 10:29:44,637 INFO sqlalchemy.engine.Engine [cached since 75.71s ago] {'name_1': 'Username: '}
Username: patrick
Username: sandy
Username: spongebob
2021-11-19 10:29:44,638 INFO sqlalchemy.engine.Engine ROLLBACK


In [82]:
# selecting with textual column Expressions: text() < "literal_columns()"
from sqlalchemy import text
stmt = (
    select(
        text("'some phrase'"), user_table.c.name
    ).order_by(user_table.c.name)
)

with engine.connect() as conn:
    print(conn.execute(stmt).all())


2021-11-19 10:35:11,138 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-19 10:35:11,139 INFO sqlalchemy.engine.Engine SELECT 'some phrase', user_account.name 
FROM user_account ORDER BY user_account.name
2021-11-19 10:35:11,140 INFO sqlalchemy.engine.Engine [cached since 202.9s ago] {}
[('some phrase', 'patrick'), ('some phrase', 'sandy'), ('some phrase', 'spongebob')]
2021-11-19 10:35:11,141 INFO sqlalchemy.engine.Engine ROLLBACK


In [83]:
from sqlalchemy import literal_column
stmt = (
    select(
        literal_column("'some phrase'").label("p"), user_table.c.name
    ).order_by(user_table.c.name)
)

# p does not contain 'single quotation'
with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(f"{row.p}, {row.name}")


2021-11-19 10:38:25,241 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-19 10:38:25,241 INFO sqlalchemy.engine.Engine SELECT 'some phrase' AS p, user_account.name 
FROM user_account ORDER BY user_account.name
2021-11-19 10:38:25,242 INFO sqlalchemy.engine.Engine [generated in 0.00147s] {}
some phrase, patrick
some phrase, sandy
some phrase, spongebob
2021-11-19 10:38:25,243 INFO sqlalchemy.engine.Engine ROLLBACK


#### The WHERE clause

In [90]:
print(user_table.c.name == 'squidward')
print(address_table.c.user_id > 10)
print(select(user_table).where(user_table.c.name == 'squidward'))


user_account.name = :name_1
address.user_id > :user_id_1
SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1


In [91]:
# two results are the same
print(
    select(address_table.c.email_address).
    where(user_table.c.name == 'squidward').
    where(address_table.c.user_id == user_table.c.id)
)

print(
    select(address_table.c.email_address).
    where(
        user_table.c.name == 'squidward',
        address_table.c.user_id == user_table.c.id
    )
)


SELECT address.email_address 
FROM address, user_account 
WHERE user_account.name = :name_1 AND address.user_id = user_account.id
SELECT address.email_address 
FROM address, user_account 
WHERE user_account.name = :name_1 AND address.user_id = user_account.id


In [93]:
# and_(), or_()
from sqlalchemy import and_, or_
print(
    select(Address.email_address).
    where(
        and_(
            or_(User.name == 'squidward', User.name == 'sandy'),
            Address.user_id == User.id
        )
    )
)


SELECT address.email_address 
FROM address, user_account 
WHERE (user_account.name = :name_1 OR user_account.name = :name_2) AND address.user_id = user_account.id


In [94]:
# for "equality(=)" comparisions, filter_by()
print(
    select(User).filter_by(name='spongebob', fullname='spongbob Squarepants')
)


SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1 AND user_account.fullname = :fullname_1


#### Explicit FROM clauses and JOINs

In [100]:
# 4 results are the same

# join_from(left side table, right side table)
print(
    select(user_table.c.name, address_table.c.email_address).
    join_from(user_table, address_table)
)

# join(right side table)
print(
    select(user_table.c.name, address_table.c.email_address).
    join(address_table)
)

# select_from(left side table).join(right side table)
print(
    select(address_table.c.email_address).
    select_from(user_table).join(address_table)
)

# setting the ON Clause : If the two tables do not have a foreignKeyConstraint, we need to use "ON"
print(
    select(address_table.c.email_address).
    select_from(user_table).
    join(address_table, user_table.c.id == address_table.c.user_id)
)


SELECT user_account.name, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id
SELECT user_account.name, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id
SELECT address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id
SELECT address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


In [98]:
# when you select only function result, use "select from"
from sqlalchemy import func
print(
    select(func.count('*')).select_from(user_table)
)


SELECT count(:count_2) AS count_1 
FROM user_account


In [102]:
# OUTER(isouter=) and FULL(full=) join
print(
    select(user_table).join(address_table, isouter=True)  # LEFT OUTER JOIN
)

print(
    select(user_table).join(address_table, full=True)  # FULL OUTER JOIN
)


SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account LEFT OUTER JOIN address ON user_account.id = address.user_id
SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account FULL OUTER JOIN address ON user_account.id = address.user_id


#### ORDER BY, GROUP BY, HAVING

In [105]:
# order_by() 
print(select(user_table).order_by(user_table.c.name))

print(select(User).order_by(User.fullname.desc()))


SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account ORDER BY user_account.name
SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account ORDER BY user_account.fullname DESC


In [106]:
# Aggregate functions with GROUP BY/ HAVING
count_fn = func.count(user_table.c.id)
print(count_fn)

count(user_account.id)


In [108]:
with engine.connect() as conn:
    result = conn.execute(
        select(User.name, func.count(Address.id).label("count")).
        join(Address).
        group_by(User.name).
        having(func.count(Address.id) > 1)
    )

print(result.all())


2021-11-19 14:22:36,665 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-19 14:22:36,665 INFO sqlalchemy.engine.Engine SELECT user_account.name, count(address.id) AS count 
FROM user_account JOIN address ON user_account.id = address.user_id GROUP BY user_account.name 
HAVING count(address.id) > %(count_1)s
2021-11-19 14:22:36,665 INFO sqlalchemy.engine.Engine [cached since 19.99s ago] {'count_1': 1}
2021-11-19 14:22:36,682 INFO sqlalchemy.engine.Engine ROLLBACK
[('sandy', 2)]


In [116]:
from sqlalchemy import func, desc
# not 'desc(func.count(Address.id))'
stmt = select(
    Address.user_id,
    func.count(Address.id).label('num_addresses')).\
    group_by("user_id").order_by("user_id", desc("num_addresses"))
print(stmt)


SELECT address.user_id, count(address.id) AS num_addresses 
FROM address GROUP BY address.user_id ORDER BY address.user_id, num_addresses DESC


#### Using Aliases

In [117]:
user_alias_1 = user_table.alias('table1')
user_alias_2 = user_table.alias('table2')

print(
    select(user_alias_1.c.name, user_alias_2.c.name).
    join_from(user_alias_1, user_alias_2,
              user_alias_1.c.id > user_alias_2.c.id)
)


SELECT table1.name, table2.name AS name_1 
FROM user_account AS table1 JOIN user_account AS table2 ON table1.id > table2.id


In [118]:
# ORM Entity Aliases: aliased()
from sqlalchemy.orm import aliased
address_alias_1 = aliased(Address)
address_alias_2 = aliased(Address)
print(
    select(User).
    join_from(User, address_alias_1).
    where(address_alias_1.email_address == 'patrick@aol.com').
    join_from(User, address_alias_2).
    where(address_alias_2.email_address == 'patrick@gmail.com')
)


SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account JOIN address AS address_1 ON user_account.id = address_1.user_id JOIN address AS address_2 ON user_account.id = address_2.user_id 
WHERE address_1.email_address = :email_address_1 AND address_2.email_address = :email_address_2


#### Subqueries and CTEs (Common Table Expression)

In [122]:
subq = select(
    func.count(address_table.c.id).label("count"),
    address_table.c.user_id
).group_by(address_table.c.user_id)

# Two results are the same.
print(subq)
print(subq.subquery())


SELECT count(address.id) AS count, address.user_id 
FROM address GROUP BY address.user_id
SELECT count(address.id) AS count, address.user_id 
FROM address GROUP BY address.user_id


In [126]:
# Question : Why Should we use 'subquery()'?
subq = select(
    func.count(address_table.c.id).label("count"),
    address_table.c.user_id
).group_by(address_table.c.user_id).subquery()
print(select(subq.c.user_id, subq.c.count))

# Answer : We can get the result but we alse get 'SADeprecationWarning'.
# When we use subquery in select function, we should use 'subquery()'


SELECT anon_1.user_id, anon_1.count 
FROM (SELECT count(address.id) AS count, address.user_id AS user_id 
FROM address GROUP BY address.user_id) AS anon_1


In [127]:
stmt = select(
    user_table.c.name,
    user_table.c.fullname,
    subq.c.count
).join_from(user_table, subq)

print(stmt)

SELECT user_account.name, user_account.fullname, anon_1.count 
FROM user_account JOIN (SELECT count(address.id) AS count, address.user_id AS user_id 
FROM address GROUP BY address.user_id) AS anon_1 ON user_account.id = anon_1.user_id


In [130]:
# CTEs : like "WITH" clause. we can use it a "recursive" style.
subq = select(
    func.count(address_table.c.id).label("count"),
    address_table.c.user_id
).group_by(address_table.c.user_id).cte()

stmt = select(
    user_table.c.name,
    user_table.c.fullname,
    subq.c.count
).join_from(user_table, subq)

print(stmt)


WITH anon_1 AS 
(SELECT count(address.id) AS count, address.user_id AS user_id 
FROM address GROUP BY address.user_id)
 SELECT user_account.name, user_account.fullname, anon_1.count 
FROM user_account JOIN anon_1 ON user_account.id = anon_1.user_id


In [154]:
# two results are the same.
# using subquery
subq = select(Address).where(~Address.email_address.like('%@aol.com'))\
    .subquery()  # NOT LIKE
print(subq)

address_subq = aliased(Address, alias=subq)  # Address object is linked to subq
stmt = select(User, address_subq).join_from(User, address_subq)\
    .order_by(User.id, address_subq.id)
with Session(engine) as session:
    for user, address in session.execute(stmt):
        print(f"{user} {address}")

# using cte
cte_obj = select(Address).where(~Address.email_address.like('%@aol.com'))\
    .cte()
address_cte = aliased(Address, cte_obj)
stmt = select(User, address_cte).join_from(User, address_cte)\
    .order_by(User.id, address_cte.id)
with Session(engine) as session:
    for user, address in session.execute(stmt):
        print(f"{user} {address}")


SELECT address.id, address.email_address, address.user_id 
FROM address 
WHERE address.email_address NOT LIKE :email_address_1
2021-11-19 16:14:28,075 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-19 16:14:28,075 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname, anon_1.id AS id_1, anon_1.email_address, anon_1.user_id 
FROM user_account JOIN (SELECT address.id AS id, address.email_address AS email_address, address.user_id AS user_id 
FROM address 
WHERE address.email_address NOT LIKE %(email_address_1)s) AS anon_1 ON user_account.id = anon_1.user_id ORDER BY user_account.id, anon_1.id
2021-11-19 16:14:28,075 INFO sqlalchemy.engine.Engine [cached since 1487s ago] {'email_address_1': '%@aol.com'}
User(id=1, name='spongebob', fullname='Spongebob Squarepants') Address(id=1, email_address='spongebob@sqlalchemy.org')
User(id=2, name='sandy', fullname='Sandy Cheeks') Address(id=2, email_address='sandy@sqlalchemy.org')
User(id=2, name='sand

#### Scalar and Correlated Subqueries

In [157]:
subq = select(func.count(address_table.c.id))\
    .where(user_table.c.id == address_table.c.user_id)
print(subq)

print(subq.scalar_subquery())  # contains '(',')'


SELECT count(address.id) AS count_1 
FROM address, user_account 
WHERE user_account.id = address.user_id
(SELECT count(address.id) AS count_1 
FROM address, user_account 
WHERE user_account.id = address.user_id)


In [167]:
subq = select(func.count(address_table.c.id))\
    .where(user_table.c.id == address_table.c.user_id).scalar_subquery()
print(subq == 5)


(SELECT count(address.id) AS count_1 
FROM address, user_account 
WHERE user_account.id = address.user_id) = :param_1


In [168]:
stmt = select(user_table.c.name, subq.label("address_count"))
print(stmt)


SELECT user_account.name, (SELECT count(address.id) AS count_1 
FROM address 
WHERE user_account.id = address.user_id) AS address_count 
FROM user_account


In [181]:
# correlate() : When we write correlated subqueries, specify "main table"
subq = select(func.count(address_table.c.id)).\
    where(user_table.c.id == address_table.c.user_id).\
    scalar_subquery().correlate(user_table)

print(subq)


(SELECT count(address.id) AS count_1 
FROM address, user_account 
WHERE user_account.id = address.user_id)


In [182]:
with engine.connect() as conn:
    result = conn.execute(
        select(
            user_table.c.name,
            address_table.c.email_address,
            subq.label("address_count")
        ).
        join_from(user_table, address_table).
        order_by(user_table.c.id, address_table.c.id)
    )
    print(result.all())


2021-11-19 16:51:39,954 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-19 16:51:39,955 INFO sqlalchemy.engine.Engine SELECT user_account.name, address.email_address, (SELECT count(address.id) AS count_1 
FROM address 
WHERE user_account.id = address.user_id) AS address_count 
FROM user_account JOIN address ON user_account.id = address.user_id ORDER BY user_account.id, address.id
2021-11-19 16:51:39,956 INFO sqlalchemy.engine.Engine [cached since 974s ago] {}
[('spongebob', 'spongebob@sqlalchemy.org', 1), ('sandy', 'sandy@sqlalchemy.org', 2), ('sandy', 'sandy@squirrelpower.org', 2)]
2021-11-19 16:51:39,957 INFO sqlalchemy.engine.Engine ROLLBACK


#### UNION, UNION ALL and other set operations

In [185]:
from sqlalchemy import union_all
stmt1 = select(user_table).where(user_table.c.name == 'sandy')
stmt2 = select(user_table).where(user_table.c.name == 'spongebob')
u = union_all(stmt1, stmt2)  # u is 'CompoundSelect' type.
with engine.connect() as conn:
    result = conn.execute(u)
    print(result.all())


2021-11-19 16:58:03,912 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-19 16:58:03,913 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = %(name_1)s UNION ALL SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = %(name_2)s
2021-11-19 16:58:03,913 INFO sqlalchemy.engine.Engine [cached since 235.7s ago] {'name_1': 'sandy', 'name_2': 'spongebob'}
[(2, 'sandy', 'Sandy Cheeks'), (1, 'spongebob', 'Spongebob Squarepants')]
2021-11-19 16:58:03,914 INFO sqlalchemy.engine.Engine ROLLBACK


In [186]:
u_subq = u.subquery()
stmt = (
    select(u_subq.c.name, address_table.c.email_address).
    join_from(address_table, u_subq).
    order_by(u_subq.c.name, address_table.c.email_address)
)
with engine.connect() as conn:
    result = conn.execute(stmt)
    print(result.all())


2021-11-19 16:59:05,511 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-19 16:59:05,512 INFO sqlalchemy.engine.Engine SELECT anon_1.name, address.email_address 
FROM address JOIN (SELECT user_account.id AS id, user_account.name AS name, user_account.fullname AS fullname 
FROM user_account 
WHERE user_account.name = %(name_1)s UNION ALL SELECT user_account.id AS id, user_account.name AS name, user_account.fullname AS fullname 
FROM user_account 
WHERE user_account.name = %(name_2)s) AS anon_1 ON anon_1.id = address.user_id ORDER BY anon_1.name, address.email_address
2021-11-19 16:59:05,513 INFO sqlalchemy.engine.Engine [cached since 179.3s ago] {'name_1': 'sandy', 'name_2': 'spongebob'}
[('sandy', 'sandy@sqlalchemy.org'), ('sandy', 'sandy@squirrelpower.org'), ('spongebob', 'spongebob@sqlalchemy.org')]
2021-11-19 16:59:05,514 INFO sqlalchemy.engine.Engine ROLLBACK


In [192]:
# selecting "ORM" Entities from Unions
stmt1 = select(User).where(User.name == 'sandy')
stmt2 = select(User).where(User.name == 'spongebob')
u = union_all(stmt1, stmt2)

print(u)

# Two results are similar, but the formats are different.
# User(id=2, name='sandy', fullname='Sandy Cheeks')
# User(id=1, name='spongebob', fullname='Spongebob Squarepants')
orm_stmt = select(User).from_statement(u)
with Session(engine) as session:
    for obj in session.execute(orm_stmt).scalars():
        print(obj)

# [(2, 'sandy', 'Sandy Cheeks'), (1, 'spongebob', 'Spongebob Squarepants')]
with Session(engine) as session:
    result = session.execute(u)
    print(result.all())


SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1 UNION ALL SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_2
2021-11-19 17:20:19,227 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-19 17:20:19,229 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = %(name_1)s UNION ALL SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = %(name_2)s
2021-11-19 17:20:19,230 INFO sqlalchemy.engine.Engine [cached since 366.1s ago] {'name_1': 'sandy', 'name_2': 'spongebob'}
User(id=2, name='sandy', fullname='Sandy Cheeks')
User(id=1, name='spongebob', fullname='Spongebob Squarepants')
2021-11-19 17:20:19,231 INFO sqlalchemy.engine.Engine ROLLBACK
2021-11-19 17:20:19,232 INFO sqlalchemy.engine.Engine BEGIN (implicit)
202

In [196]:
user_alias = aliased(User, u.subquery())
orm_stmt = select(user_alias).order_by(user_alias.id)

# three results are different each other.
with Session(engine) as session:
    for obj in session.execute(orm_stmt).scalars():
        print(obj)
# User(id=1, name='spongebob', fullname='Spongebob Squarepants')
# User(id=2, name='sandy', fullname='Sandy Cheeks')

# Question : if we omit 'scalars()', what happened? not separate element.
with Session(engine) as session:
    for obj in session.execute(orm_stmt):
        print(obj)
# (User(id=1, name='spongebob', fullname='Spongebob Squarepants'),)
# (User(id=2, name='sandy', fullname='Sandy Cheeks'),)

with Session(engine) as session:
    print(session.execute(orm_stmt))
# <sqlalchemy.engine.result.ChunkedIteratorResult object at 0x000001CBB91EB760>


2021-11-19 17:29:30,445 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-19 17:29:30,446 INFO sqlalchemy.engine.Engine SELECT anon_1.id, anon_1.name, anon_1.fullname 
FROM (SELECT user_account.id AS id, user_account.name AS name, user_account.fullname AS fullname 
FROM user_account 
WHERE user_account.name = %(name_1)s UNION ALL SELECT user_account.id AS id, user_account.name AS name, user_account.fullname AS fullname 
FROM user_account 
WHERE user_account.name = %(name_2)s) AS anon_1 ORDER BY anon_1.id
2021-11-19 17:29:30,447 INFO sqlalchemy.engine.Engine [cached since 274.7s ago] {'name_1': 'sandy', 'name_2': 'spongebob'}
User(id=1, name='spongebob', fullname='Spongebob Squarepants')
User(id=2, name='sandy', fullname='Sandy Cheeks')
2021-11-19 17:29:30,449 INFO sqlalchemy.engine.Engine ROLLBACK
2021-11-19 17:29:30,450 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-19 17:29:30,450 INFO sqlalchemy.engine.Engine SELECT anon_1.id, anon_1.name, anon_1.fullname 
FROM (SELECT 

#### EXISTS subqueries

In [9]:
# subq is 'EXISTS' object
from sqlalchemy import select, func

subq = (
    select(func.count(address_table.c.id)).
    where(user_table.c.id == address_table.c.user_id).
    group_by(address_table.c.user_id).
    having(func.count(address_table.c.id) > 1)
).exists()

print(subq)

with engine.connect() as conn:
    result = conn.execute(
        select(user_table.c.name).where(subq)
    )
    print(result.all())

EXISTS (SELECT count(address.id) AS count_1 
FROM address, user_account 
WHERE user_account.id = address.user_id GROUP BY address.user_id 
HAVING count(address.id) > :count_2)
2021-11-22 08:10:42,589 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-22 08:10:42,590 INFO sqlalchemy.engine.Engine SELECT user_account.name 
FROM user_account 
WHERE EXISTS (SELECT count(address.id) AS count_1 
FROM address 
WHERE user_account.id = address.user_id GROUP BY address.user_id 
HAVING count(address.id) > %(count_2)s)
2021-11-22 08:10:42,591 INFO sqlalchemy.engine.Engine [generated in 0.00132s] {'count_2': 1}
[('sandy',)]
2021-11-22 08:10:42,617 INFO sqlalchemy.engine.Engine ROLLBACK


In [10]:
subq = (
    select(address_table.c.id).
    where(user_table.c.id == address_table.c.user_id)
).exists()

with engine.connect() as conn:
    result = conn.execute(
        select(user_table.c.name).where(~subq)
    )
    print(result.all())

2021-11-22 08:18:01,918 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-22 08:18:01,919 INFO sqlalchemy.engine.Engine SELECT user_account.name 
FROM user_account 
WHERE NOT (EXISTS (SELECT address.id 
FROM address 
WHERE user_account.id = address.user_id))
2021-11-22 08:18:01,919 INFO sqlalchemy.engine.Engine [generated in 0.00158s] {}
[('patrick',)]
2021-11-22 08:18:01,928 INFO sqlalchemy.engine.Engine ROLLBACK


#### Working with SQL Functions

In [11]:
# count()
print(select(func.count()).select_from(user_table))

SELECT count(*) AS count_1 
FROM user_account


In [12]:
# lower()
print(select(func.lower("A String With Much UPPERCASE")))

SELECT lower(:lower_2) AS lower_1


In [13]:
# now()
stmt = select(func.now())
with engine.connect() as conn:
    result = conn.execute(stmt)
    print(result.all())

2021-11-22 08:22:33,378 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-22 08:22:33,379 INFO sqlalchemy.engine.Engine SELECT now() AS now_1
2021-11-22 08:22:33,380 INFO sqlalchemy.engine.Engine [generated in 0.00209s] {}
[(datetime.datetime(2021, 11, 22, 8, 22, 33, 380602, tzinfo=datetime.timezone(datetime.timedelta(seconds=32400))),)]
2021-11-22 08:22:33,382 INFO sqlalchemy.engine.Engine ROLLBACK


In [16]:
# func : any name that is accessed from this namespace is automatically considered to be a SQL function
print(select(func.some_crazy2_function(user_table.c.name, 17)))

SELECT some_crazy2_function(user_account.name, :some_crazy2_function_2) AS some_crazy2_function_1 
FROM user_account


In [19]:
# dialect
from sqlalchemy.dialects import postgresql, oracle
print(select(func.now()).compile(dialect=postgresql.dialect()))
print(select(func.now()).compile(dialect=oracle.dialect()))

SELECT now() AS now_1
SELECT CURRENT_TIMESTAMP AS now_1 FROM DUAL


In [21]:
# functions have return types
func.now().type

DateTime()

In [22]:
from sqlalchemy import JSON
function_expr = func.json_object('{a, 1, b, "def", c, 3.5}', type_=JSON)
stmt = select(function_expr["def"])
print(stmt)

SELECT json_object(:json_object_1)[:json_object_2] AS anon_1


In [23]:
# Built-in functions have pre-configured return types
m1 = func.max(Column("some_int", Integer))
m1.type

Integer()

In [24]:
m2 = func.max(Column("some_str", String))
m2.type

String()

In [26]:
func.now().type

DateTime()

In [27]:
func.current_date().type

Date()

In [29]:
func.concat("x","y").type

String()

In [30]:
# However, lower() and upper() have a "null" return type.
func.upper("lowercase").type

NullType()

In [31]:
# using window functions
# over(partition_by=)
stmt = select(
    func.row_number().over(partition_by=user_table.c.name),
    user_table.c.name,
    address_table.c.email_address
).select_from(user_table).join(address_table)

with engine.connect() as conn:
    result = conn.execute(stmt)
    print(result.all())


2021-11-22 08:52:55,151 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-22 08:52:55,151 INFO sqlalchemy.engine.Engine SELECT row_number() OVER (PARTITION BY user_account.name) AS anon_1, user_account.name, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id
2021-11-22 08:52:55,151 INFO sqlalchemy.engine.Engine [generated in 0.00240s] {}
[(1, 'sandy', 'sandy@sqlalchemy.org'), (2, 'sandy', 'sandy@squirrelpower.org'), (1, 'spongebob', 'spongebob@sqlalchemy.org')]
2021-11-22 08:52:55,151 INFO sqlalchemy.engine.Engine ROLLBACK


In [32]:
# over(order_by=)
stmt = select(
    func.count().over(order_by=user_table.c.name),
    user_table.c.name,
    address_table.c.email_address).select_from(user_table).join(address_table)

with engine.connect() as conn:
    result = conn.execute(stmt)
    print(result.all())


2021-11-22 08:57:56,084 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-22 08:57:56,086 INFO sqlalchemy.engine.Engine SELECT count(*) OVER (ORDER BY user_account.name) AS anon_1, user_account.name, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id
2021-11-22 08:57:56,086 INFO sqlalchemy.engine.Engine [generated in 0.00174s] {}
[(2, 'sandy', 'sandy@sqlalchemy.org'), (2, 'sandy', 'sandy@squirrelpower.org'), (3, 'spongebob', 'spongebob@sqlalchemy.org')]
2021-11-22 08:57:56,088 INFO sqlalchemy.engine.Engine ROLLBACK


In [33]:
# special modifiers WITHIN GROUP, FILTER
# unnest : an array to a set of rows
print(
    func.unnest(
        func.percentitle_disc([0.25,0.5,0.75,1]).within_group(user_table.c.name)
    )
)


unnest(percentitle_disc(:percentitle_disc_1) WITHIN GROUP (ORDER BY user_account.name))


In [34]:
stmt = select(
    func.count(address_table.c.email_address).filter(user_table.c.name == 'sandy'),
    func.count(address_table.c.email_address).filter(user_table.c.name == 'spongebob')
).select_from(user_table).join(address_table)

with engine.connect() as conn:
    result = conn.execute(stmt)
    print(result.all())


2021-11-22 09:17:13,851 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-22 09:17:13,852 INFO sqlalchemy.engine.Engine SELECT count(address.email_address) FILTER (WHERE user_account.name = %(name_1)s) AS anon_1, count(address.email_address) FILTER (WHERE user_account.name = %(name_2)s) AS anon_2 
FROM user_account JOIN address ON user_account.id = address.user_id
2021-11-22 09:17:13,854 INFO sqlalchemy.engine.Engine [generated in 0.00288s] {'name_1': 'sandy', 'name_2': 'spongebob'}
[(2, 1)]
2021-11-22 09:17:13,862 INFO sqlalchemy.engine.Engine ROLLBACK


In [168]:
from sqlalchemy import create_engine
engine2 = create_engine("sqlite+pysqlite:///:memory:", echo=True, future=True)

In [189]:
# Table-Valued Functions : I changed 'json_each' to 'json_array_elements_text' because some errors are printed out.
# onetwothree = func.json_each('["one", "two", "three"]').table_valued("value")  
onetwothree = func.json_array_elements_text('["one", "two", "three"]').table_valued("value")
stmt = select(onetwothree).where(onetwothree.c.value.in_(["two", "three"]))
with engine.connect() as conn:  
    result = conn.execute(stmt)
    print(result.all())

2021-11-22 13:26:57,896 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-22 13:26:57,897 INFO sqlalchemy.engine.Engine SELECT anon_1.value 
FROM json_array_elements_text(%(json_array_elements_text_1)s) AS anon_1 
WHERE anon_1.value IN (%(value_1_1)s, %(value_1_2)s)
2021-11-22 13:26:57,898 INFO sqlalchemy.engine.Engine [cached since 953.5s ago] {'json_array_elements_text_1': '["one", "two", "three"]', 'value_1_1': 'two', 'value_1_2': 'three'}
[('two',), ('three',)]
2021-11-22 13:26:57,899 INFO sqlalchemy.engine.Engine ROLLBACK


In [149]:
# column_valued : using in the FROM clause
stmt = select(func.json_array_elements('["one", "two"]').column_valued("x"))
print(stmt)

SELECT x 
FROM json_array_elements(:json_array_elements_1) AS x


In [150]:
from sqlalchemy.dialects import postgresql
stmt = select(func.scalar_strings(5).column_valued("s"))
print(stmt.compile(dialect=postgresql.dialect()))

SELECT s 
FROM scalar_strings(%(scalar_strings_1)s) AS s


### Updating and Deleting Rows with Core

#### The update() SQL Expression Construct

In [191]:
from sqlalchemy import update
stmt = (
    update(user_table).where(user_table.c.name == 'patrick').
    values(fullname='Patrick the Star')
)
print(stmt)

UPDATE user_account SET fullname=:fullname WHERE user_account.name = :name_1


In [193]:
stmt = (
    update(user_table).
    values(fullname="Username: " + user_table.c.name)
)
print(stmt)

UPDATE user_account SET fullname=(:name_1 || user_account.name)


In [192]:
from sqlalchemy import bindparam
stmt = (
    update(user_table).
    where(user_table.c.name == bindparam('oldname')).
    values(name=bindparam('newname'))
)

with engine.begin() as conn:
    conn.execute(
        stmt,
        [
            {'oldname': 'jack', 'newname': 'ed'},
            {'oldname': 'wendy', 'newname': 'mary'},
            {'oldname': 'jim', 'newname': 'jake'},
        ]
    )


2021-11-22 13:48:47,749 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-22 13:48:47,751 INFO sqlalchemy.engine.Engine UPDATE user_account SET name=%(newname)s WHERE user_account.name = %(oldname)s
2021-11-22 13:48:47,752 INFO sqlalchemy.engine.Engine [generated in 0.00057s] ({'newname': 'ed', 'oldname': 'jack'}, {'newname': 'mary', 'oldname': 'wendy'}, {'newname': 'jake', 'oldname': 'jim'})
2021-11-22 13:48:47,754 INFO sqlalchemy.engine.Engine COMMIT


In [198]:
# correlated updates
scalar_subq = (
    select(address_table.c.email_address)
    .where(address_table.c.user_id == user_table.c.id)
    .order_by(address_table.c.id)
    .limit(1)  # like ROWNUM <= 1
    .scalar_subquery()
)

update_stmt = update(user_table).values(fullname=scalar_subq)
print(update_stmt)


UPDATE user_account SET fullname=(SELECT address.email_address 
FROM address 
WHERE address.user_id = user_account.id ORDER BY address.id
 LIMIT :param_1)


In [199]:
# UPDATE..FROM : other tables can be used for choosing update target.
update_stmt = (
    update(user_table)
    .where(user_table.c.id == address_table.c.user_id)
    .where(address_table.c.email_address == 'patrick@aol.com')
    .values(fullname='Pat')
)

print(update_stmt)


UPDATE user_account SET fullname=:fullname FROM address WHERE user_account.id = address.user_id AND address.email_address = :email_address_1


#### The delete() SQL Expression Construct

In [200]:
from sqlalchemy import delete
stmt = delete(user_table).where(user_table.c.name == 'patrick')
print(stmt)

DELETE FROM user_account WHERE user_account.name = :name_1


#### Getting Affected Row Count from UPDATE, DELETE : rowcount

In [203]:
with engine.begin() as conn:
    result = conn.execute(
        update(user_table).values(fullname="Patrick McStar")
        .where(user_table.c.name == 'patrick')
    )
    print(result.rowcount)

# IF the rows were modified or not, # of rows "matched by the WHERE" clause.

2021-11-22 14:10:49,250 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-22 14:10:49,251 INFO sqlalchemy.engine.Engine UPDATE user_account SET fullname=%(fullname)s WHERE user_account.name = %(name_1)s
2021-11-22 14:10:49,252 INFO sqlalchemy.engine.Engine [cached since 231.3s ago] {'fullname': 'Patrick McStar', 'name_1': 'patrick'}
1
2021-11-22 14:10:49,253 INFO sqlalchemy.engine.Engine COMMIT


#### Using RETURNING with UPDATE, DELETE

In [205]:
update_stmt = (
    update(user_table)
    .values(fullname='Patrick the Star')
    .where(user_table.c.name == 'Patrick')
    .returning(user_table.c.id, user_table.c.name)
)
print(update_stmt)

UPDATE user_account SET fullname=:fullname WHERE user_account.name = :name_1 RETURNING user_account.id, user_account.name


In [206]:
delete_stmt = (
    delete(user_table)
    .where(user_table.c.name == 'Patrick')
    .returning(user_table.c.id, user_table.c.name)
)
print(delete_stmt)

DELETE FROM user_account WHERE user_account.name = :name_1 RETURNING user_account.id, user_account.name
